# R-CNN Inference: Complete Detection Pipeline

Pipeline: Selective Search → CNN Features → SVM Classification → NMS → Bbox Regression

Combines all 4 trained stages:
1. Fine-tuned AlexNet
2. Linear SVM classifiers
3. Bounding box regressors
4. Non-maximum suppression

In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
import random

from rcnn.config.config import RCNNConfig
from rcnn.inference import RCNNDetector

In [ ]:
ANNOTATIONS_CSV = Path("data/airbus/annotations/train_annotations_filtered.csv")
IMAGES_DIR = Path("data/airbus/images/train")
ARTIFACTS_DIR = Path("artifacts/rcnn")

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
elif torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    print("Using Apple Silicon GPU")
else:
    DEVICE = torch.device("cpu")
    print("Using CPU")

In [ ]:
config = RCNNConfig()
config.output_dir = ARTIFACTS_DIR

detector = RCNNDetector(config, device=str(DEVICE))
detector.load_models(ARTIFACTS_DIR)

## Run Detection on Sample Images

In [ ]:
annotations = pd.read_csv(ANNOTATIONS_CSV)
sample_image_ids = random.sample(list(annotations['image_id'].unique()), 3)

for image_id in sample_image_ids:
    image_path = IMAGES_DIR / image_id
    img = Image.open(image_path).convert("RGB")
    
    print(f"\nProcessing: {image_id}")
    
    detections = detector.detect(
        img,
        score_threshold=0.5,
        nms_threshold=0.3,
        use_bbox_regression=True
    )
    
    print(f"Found {len(detections)} detections")
    
    draw = ImageDraw.Draw(img)
    
    for det in detections:
        bbox = det['bbox']
        score = det['score']
        class_id = det['class_id']
        
        draw.rectangle(bbox, outline="red", width=3)
        
        text = f"Class {class_id}: {score:.2f}"
        draw.text((bbox[0], bbox[1] - 15), text, fill="red")
    
    plt.figure(figsize=(12, 8))
    plt.imshow(img)
    plt.title(f"{image_id} - {len(detections)} detections")
    plt.axis('off')
    plt.show()

## Compare: With vs Without Bbox Regression

In [ ]:
sample_image_id = random.choice(annotations['image_id'].unique())
image_path = IMAGES_DIR / sample_image_id
img = Image.open(image_path).convert("RGB")

detections_no_reg = detector.detect(img, score_threshold=0.5, nms_threshold=0.3, use_bbox_regression=False)
detections_with_reg = detector.detect(img, score_threshold=0.5, nms_threshold=0.3, use_bbox_regression=True)

fig, axes = plt.subplots(1, 2, figsize=(20, 10))

img_no_reg = img.copy()
draw_no_reg = ImageDraw.Draw(img_no_reg)
for det in detections_no_reg:
    bbox = det['bbox']
    score = det['score']
    draw_no_reg.rectangle(bbox, outline="blue", width=3)
    draw_no_reg.text((bbox[0], bbox[1] - 15), f"{score:.2f}", fill="blue")

axes[0].imshow(img_no_reg)
axes[0].set_title(f"Without Bbox Regression ({len(detections_no_reg)} detections)")
axes[0].axis('off')

img_with_reg = img.copy()
draw_with_reg = ImageDraw.Draw(img_with_reg)
for det in detections_with_reg:
    bbox = det['bbox']
    score = det['score']
    draw_with_reg.rectangle(bbox, outline="red", width=3)
    draw_with_reg.text((bbox[0], bbox[1] - 15), f"{score:.2f}", fill="red")

axes[1].imshow(img_with_reg)
axes[1].set_title(f"With Bbox Regression ({len(detections_with_reg)} detections)")
axes[1].axis('off')

plt.tight_layout()
plt.show()

## Detection Statistics

In [ ]:
all_detections = []
num_test_images = 20

test_image_ids = random.sample(list(annotations['image_id'].unique()), num_test_images)

for image_id in test_image_ids:
    image_path = IMAGES_DIR / image_id
    img = Image.open(image_path).convert("RGB")
    
    detections = detector.detect(img, score_threshold=0.5, nms_threshold=0.3, use_bbox_regression=True)
    all_detections.extend(detections)

print(f"\nDetection Statistics (on {num_test_images} images):")
print(f"Total detections: {len(all_detections)}")
print(f"Average detections per image: {len(all_detections) / num_test_images:.2f}")

if len(all_detections) > 0:
    scores = [det['score'] for det in all_detections]
    print(f"Score statistics:")
    print(f"  Mean: {np.mean(scores):.3f}")
    print(f"  Min: {np.min(scores):.3f}")
    print(f"  Max: {np.max(scores):.3f}")
    
    plt.figure(figsize=(10, 5))
    plt.hist(scores, bins=20, edgecolor='black')
    plt.xlabel('Detection Score')
    plt.ylabel('Count')
    plt.title('Distribution of Detection Scores')
    plt.grid(True, alpha=0.3)
    plt.show()

## Save Detection Results

In [ ]:
results_dir = Path("results/rcnn_detections")
results_dir.mkdir(parents=True, exist_ok=True)

for image_id in test_image_ids[:5]:
    image_path = IMAGES_DIR / image_id
    img = Image.open(image_path).convert("RGB")
    
    detections = detector.detect(img, score_threshold=0.5, nms_threshold=0.3, use_bbox_regression=True)
    
    draw = ImageDraw.Draw(img)
    for det in detections:
        bbox = det['bbox']
        score = det['score']
        draw.rectangle(bbox, outline="red", width=3)
        draw.text((bbox[0], bbox[1] - 15), f"{score:.2f}", fill="red")
    
    output_path = results_dir / f"detection_{image_id}"
    img.save(output_path)
    print(f"Saved: {output_path}")

print(f"\nSaved {min(5, len(test_image_ids))} detection results to {results_dir}")